In [6]:
# Import the required libraries
import pandas as pd
import numpy as np
from pathlib import Path
import hvplot.pandas

In [7]:
# Read in Crypto Data
File = Path('../Dataset/crypto_df.csv')
crypto_df = pd.read_csv(File)
crypto_df.head()



,date,BTC,ETH,XRP,BNB,SOL,ADA,MATIC,SAND,MANA,ALGO
0,2021-12-03,76880.95,6053.03,1.3218,854.1,304.42,2.230,3.262,8.6267,3.9031,1.7468
1,2021-12-04,71377.71,5965.77,1.2267,827.8,291.83,2.065,2.869,8.8031,4.1199,1.6491
2,2021-12-05,70677.10,6001.06,1.1507,798.4,279.51,1.973,2.923,7.7280,3.6862,1.7650
3,2021-12-06,71800.02,6185.37,1.1750,836.7,275.23,2.027,3.247,7.8423,3.7677,1.7629
4,2021-12-07,71190.56,6060.44,1.1498,814.6,268.03,1.941,3.305,7.5142,3.8132,1.6852


In [8]:
# Create new DF
btc_df = crypto_df[['date', 'BTC']]

# Calculate the hourly returns using the closing prices and the pct_change function
btc_df["actual_returns"] = btc_df["BTC"].pct_change()
btc_df = btc_df.dropna()


C:\Users\Sam\AppData\Local\Temp\ipykernel_20732\343327718.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  btc_df["actual_returns"] = btc_df["BTC"].pct_change()


In [9]:
# Define a window size of 50 & 100
short_window = 50
long_window = 100

# Create a simple moving average (SMA) using the short_window and assign this to a new columns called sma_fast
btc_df["sma_50"] = btc_df["BTC"].rolling(window=short_window).mean()
btc_df["sma_100"] = btc_df["BTC"].rolling(window=long_window).mean()
btc_df = btc_df.dropna()



C:\Users\Sam\AppData\Local\Temp\ipykernel_20732\4144075867.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  btc_df["sma_50"] = btc_df["BTC"].rolling(window=short_window).mean()
C:\Users\Sam\AppData\Local\Temp\ipykernel_20732\4144075867.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  btc_df["sma_100"] = btc_df["BTC"].rolling(window=long_window).mean()


In [10]:
# Create a new column in the btc_df called signal setting its value to zero.
btc_df["Signal"] = 0.0

# Generate the trading signal 0 or 1,
# where 1 is the short-window (SMA50) is less than the long-window (SMA100)
btc_df["Signal"][short_window:] = np.where(
    btc_df["sma_50"][short_window:] < btc_df["sma_100"][short_window:], 1.0, 0.0
)

# Review the DataFrame
btc_df = btc_df.dropna()
btc_df

C:\Users\Sam\AppData\Local\Temp\ipykernel_20732\2536325636.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  btc_df["Signal"] = 0.0


,date,BTC,actual_returns,sma_50,sma_100,Signal
100,2022-03-13,51881.12,-0.027076,55698.3108,59921.9614,0.0
101,2022-03-14,54900.68,0.058202,55783.2966,59757.1911,0.0
102,2022-03-15,54578.87,-0.005862,55846.8212,59596.2088,0.0
103,2022-03-16,56486.20,0.034946,55940.8736,59443.0706,0.0
104,2022-03-17,55565.25,-0.016304,56015.8906,59286.8175,0.0
...,...,...,...,...,...,...
175,2022-05-27,39968.24,-0.027654,49763.4210,53268.5316,1.0
176,2022-05-28,40609.29,0.016039,49437.9008,53109.1439,1.0
177,2022-05-29,41139.43,0.013055,49111.2754,52961.7451,1.0
178,2022-05-30,44108.49,0.072171,48857.9970,52843.0183,1.0


In [11]:
# Calculate the points in time when the Signal value changes
# Identify trade entry (1) and exit (-1) points
btc_df["Entry/Exit"] = btc_df["Signal"].diff()

In [12]:
# Visualize exit position relative to close price
exit = btc_df[btc_df["Entry/Exit"] == -1.0]["BTC"].hvplot.scatter(
    color="red",
    marker="v",
    size=200,
    legend=False,
    ylabel="Price in $",
    width=1000,
    height=400)


# Visualize entry position relative to close price
entry = btc_df[btc_df["Entry/Exit"] == 1.0]["BTC"].hvplot.scatter(
    color="green",
    marker="^",
    size=200,
    legend=False,
    ylabel="Price in $",
    width=1000,
    height=400)


# Visualize close price for the investment
security_close = btc_df[["BTC"]].hvplot(
    line_color="lightgray",
    ylabel="Price in $",
    width=1000,
    height=400)

# Visualize moving averages
moving_avgs = btc_df[["SMA50", "SMA100"]].hvplot(
    ylabel="Price in $",
    width=1000,
    height=400)

KeyError: "None of [Index(['SMA50', 'SMA100'], dtype='object')] are in the [columns]"

In [ ]:
# Create the overlay plot
entry_exit_plot = security_close * moving_avgs * entry * exit

# Show the plot
entry_exit_plot.opts(
    title="Bitcoin - SMA50, SMA100, Entry and Exit Points"
)

In [ ]:
# Initialize trade_type column for buys and sells
btc_df['trade_type'] = np.nan

# Initialize variable to hold the previous_price
previous_price = 0
tw_sentiment = 0 

# Loop through the Pandas DataFrame and code the conditions of the trading strategy 
for index, row in btc_df.iterrows():
    
    # buy if the previous price is 0 and Twitter Sentiment has Positive for 7 consecutive days.
    if previous_price == 0: #and sum(tw_sentiment == 7):
        btc_df.loc[index, "trade_type"] = "buy"

    # buy if the current day price is less than the previous day price and Twitter Sentiment has Positive for 4 consecutive days
    elif row["BTC"] < previous_price: #and sum(tw_sentiment == 4):
        btc_df.loc[index, "trade_type"] = "buy"

    # sell if the current day price is greater than the previous day price and Twitter Sentiment has Negative for 3 consecutive days
    elif row["BTC"] > previous_price: # and sum(tw_sentiment < 3):
        btc_df.loc[index, "trade_type"] = "sell"

    # else hold if the current day price is equal to the previous day price and Twitter Sentiment is Neutrual or Greater in 4 days
    else:
        btc_df.loc[index, "trade_type"] = "hold"
    
    # reset the previous_price variable to the close price of the current row
    previous_price = row["BTC"]

    # if the index is the last index of the Dataframe, set the trade_type to sell
    if index == btc_df.index[-1]:
        btc_df.loc[index, "trade_type"] = "sell"


btc_df.head()

C:\Users\Sam\AppData\Local\Temp\ipykernel_23476\2084711459.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  btc_df['trade_type'] = np.nan
C:\Users\Sam\AppData\Local\Temp\ipykernel_23476\2084711459.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  btc_df.loc[index, "trade_type"] = "buy"


,date,BTC,trade_type
0,2021-12-03,76880.95,buy
1,2021-12-04,71377.71,buy
2,2021-12-05,70677.10,buy
3,2021-12-06,71800.02,sell
4,2021-12-07,71190.56,buy


In [ ]:
# Create a Sell Plot
sell = btc_df[btc_df["trade_type"] == "sell"]["BTC"].hvplot.scatter(
    color="red",
    marker="v",
    size=200,
    legend=False,
    ylabel="Price in $",
    width=1000,
    height=400)

# Create a Buy Plot
buy = btc_df[btc_df["trade_type"] == "buy"]["BTC"].hvplot.scatter(
    color="green",
    marker="v",
    size=200,
    legend=False,
    ylabel="Price in $",
    width=1000,
    height=400)

# Create a Hold Plot
neu = btc_df[btc_df["trade_type"] == "hold"]["BTC"].hvplot.scatter(
    color="orange",
    marker="v",
    size=200,
    legend=False,
    ylabel="Price in $",
    width=1000,
    height=400) 

security_close = btc_df[['BTC']].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400)    

# Create the overlay plot
buy_sell_plot = buy * sell * neu * security_close

# Show the plot
buy_sell_plot.opts(
    title="Bitcoin - Buy, Sell and Hold Chart"
)     

:Overlay
   .Scatter.BTC.I   :Scatter   [index]   (BTC)
   .Scatter.BTC.II  :Scatter   [index]   (BTC)
   .Scatter.BTC.III :Scatter   [index]   (BTC)
   .Curve.I         :Curve   [index]   (BTC)

In [ ]:
crypto_df = crypto_df.set_index("date")
crypto_df = crypto_df.stack()
crypto_df.head()

# Review the DataFrame
short_window = 50
long_window = 100

    #Generate the short and long moving averages (50 and 100 days, respectively)\n",
    sma_df = crypto_df.copy()
  
    for col in sma_df.columns:
        if col != 'date':,
            slow_col_nam = col +'_'+ 'SMA100',
            fast_col_name = col +'_'+ 'SMA50',
            sma_df = sma_df
            sma_df[slow_col_nam] = sma_df[col].rolling(window=long_window).mean(),
            sma_df[fast_col_name] =  sma_df[col].rolling(window=short_window).mean(),
            sma_df = sma_df.drop(columns = [col]),
    

    # Review the DataFrame\n",
    sma_df.head(2000)
    
    # Generate the trading signal 0 or 1,\n",
    # where 1 is the short-window (SMA50) is less than the long-window (SMA100)\n",
    #for col in sma_df.columns:\n",
        #if col != 'date':\n",
            #new_col_name = col + \"_\" + \"Signal\"\n",
            #sma_50 = ['SMA50' in i for i in sma_df.columns]\n",
            #sma_100 = ['SMA100' in i for i in sma_df.columns]\n",
            #sma_df[new_col_name][short_window:] = np.where(\n",
            #sma_df[sma_50][short_window:] < sma_df[sma_100][short_window:]\n",


IndentationError: unexpected indent (4270126934.py, line 10)